In [1]:
#Import modules
import psycopg2
import pandas as pd
import numpy as np

#Import password for postgres
from passwords import password 

In [2]:
#Open a connection to postgres and create a connection object
conn = psycopg2.connect(host='localhost', dbname='glass', user='postgres', password=password)
#Create a cursor object to execute commands
cur = conn.cursor()

cur.execute('DROP TABLE IF EXISTS glass_data')
cur.execute('''
    CREATE TABLE glass_data(
        RI float,
        NA float,
        MG float,
        AL float,
        SI float,
        K float,
        CA float,
        BA float,
        FE float,
        TYPE integer)
''')

#Commit changes 
conn.commit()

In [3]:
#Create a cursor object to execute commands
cur = conn.cursor()

#Open the created csv file
with open('glass (1).csv', 'r') as i:
    #Skip header row
    next(i) 
    
    #Call the created unemployment table and copy data from csv file into table
    cur.copy_from(i, 'glass_data', sep=',', null='')

#Commit changes
#cur.execute('ALTER TABLE observation_data DROP COLUMN TYPE')
conn.commit()

In [4]:
cur = conn.cursor()

cur.execute('''
    DROP TABLE IF EXISTS observation_data;
    CREATE TABLE observation_data (
        OBSERVATION_ID SERIAL PRIMARY KEY,
        RI float,
        NA float,
        MG float,
        AL float,
        SI float,
        K float,
        CA float,
        BA float,
        FE float);
    INSERT INTO observation_data (RI, NA, MG, AL, SI, K, CA, BA, FE)
    SELECT RI, NA, MG, AL, SI, K, CA, BA, FE from glass_data;''')

conn.commit()

In [5]:
cur.execute('DROP TABLE IF EXISTS observation_class')
cur.execute('''
    CREATE TABLE observation_class(
        OBSERVATION_ID SERIAL PRIMARY KEY,
        CLASS_ID integer);
    INSERT INTO observation_class (CLASS_ID) SELECT (TYPE) FROM glass_data;
    DROP TABLE glass_data''')

#Commit changes 
conn.commit()

In [6]:
cur.execute('DROP TABLE IF EXISTS class_information')
cur.execute('''
    CREATE TABLE class_information(
        CLASS_ID SERIAL PRIMARY KEY,
        CLASS_DESCRIPTION varchar);
    INSERT INTO class_information (CLASS_DESCRIPTION) 
    VALUES ('building_windows_float_processed'), ('building_windows_non_float_processed'), 
    ('vehicle_windows_float_processed'), ('vehicle_windows_non_float_processed (none in this database)'), 
    ('containers'), ('tableware'), ('headlamps');''')

#Commit changes 
conn.commit()
#Close connection
conn.close()

## Queries

In [ ]:
#Open a connection to postgres and create a connection object
conn = psycopg2.connect(host='localhost', dbname='postgres', user='postgres', password=password)

query = '''
    SELECT (MIN(ri), MAX(ri)) AS range_ri, (MIN(na), MAX(na)) AS range_na, (MIN(mg), MAX(mg)) AS range_mg,
        (MIN(al), MAX(al)) AS range_al, (MIN(si), MAX(si)) AS range_si, (MIN(k), MAX(k)) AS range_k,
        (MIN(ca), MAX(ca)) AS range_ca, (MIN(ba), MAX(ba)) AS range_ba, (MIN(fe), MAX(fe)) AS range_fe
    FROM observation_data d
    JOIN observation_class c ON d.observation_id = c.observation_id
    GROUP BY c.class_id;'''

#Reads sql query into a dataframe
pd.read_sql(query, conn)

In [ ]:
query = '''
    SELECT MAX(ri) as max_ri, MAX(na) AS max_na, MAX(mg) AS max_mg, MAX(al) AS max_al, MAX(si) AS max_si, 
        MAX(k) AS max_k, MAX(ca) AS max_ca, MAX(ba) AS max_ba, MAX(fe) AS max_fe
    FROM observation_data d
    JOIN observation_class c ON d.observation_id = c.observation_id
    GROUP BY c.class_id;'''

#Reads sql query into a dataframe
pd.read_sql(query, conn)

In [ ]:
query = '''
    SELECT MIN(ri) as min_ri, MIN(na) AS min_na, MIN(mg) AS min_mg, MIN(al) AS min_al, MIN(si) AS min_si, 
        MIN(k) AS min_k, MIN(ca) AS min_ca, MIN(ba) AS min_ba, MIN(fe) AS min_fe
    FROM observation_data d
    JOIN observation_class c ON d.observation_id = c.observation_id
    GROUP BY c.class_id;'''

#Reads sql query into a dataframe
pd.read_sql(query, conn)

In [ ]:
#Close connection
conn.close()